In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
datosML=pd.read_csv("D:\Bootcamp\MLproyecto\Data\Processed\datosML.csv",index_col=False)
datosML

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,tipodeobra
0,2,0,4.0,1,1
1,3,0,2.0,2,2
2,3,0,4.0,2,1
3,4,0,2.0,2,4
4,4,0,1.0,3,1
...,...,...,...,...,...
1332,4,0,5.0,1,1
1333,4,0,5.0,9,1
1334,4,0,5.0,1,1
1335,4,0,5.0,3,2


In [4]:
datosML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MES            1337 non-null   int64  
 1   Resultado      1337 non-null   int64  
 2   COMERCIAL_NUM  1337 non-null   float64
 3   TELEOP_NUM     1337 non-null   int64  
 4   tipodeobra     1337 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 52.4 KB


In [5]:
datosML.describe()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,tipodeobra
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,5.899776,0.225879,3.382947,3.305161,3.359761
std,3.392993,0.418316,1.866810,2.782470,1.606022
min,1.000000,0.000000,1.000000,1.000000,1.000000
25%,3.000000,0.000000,2.000000,1.000000,2.000000
50%,6.000000,0.000000,3.000000,2.000000,4.000000
75%,9.000000,0.000000,5.000000,6.000000,5.000000
max,12.000000,1.000000,7.000000,11.000000,5.000000


In [6]:
datosML.corr()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,tipodeobra
MES,1.000000,-0.016207,0.043288,-0.071522,0.086977
Resultado,-0.016207,1.000000,0.091392,-0.018108,-0.028575
COMERCIAL_NUM,0.043288,0.091392,1.000000,-0.029575,-0.027511
TELEOP_NUM,-0.071522,-0.018108,-0.029575,1.000000,-0.118888
tipodeobra,0.086977,-0.028575,-0.027511,-0.118888,1.000000


# Comprobación de equilibrio de target

In [7]:
datosML["Resultado"].value_counts()

0    1035
1     302
Name: Resultado, dtype: int64

In [8]:
#Dado que se se observa un desequilibrio en nuestro target, realizaremos un oversampling para solucionarlo

from imblearn.over_sampling import RandomOverSampler

oversampler = RandomOverSampler()
X_oversampled, y_oversampled = oversampler.fit_resample(datosML.drop("Resultado", axis=1), datosML["Resultado"])

# Creamos un DataFrame con el oversampling
datosoversampled = pd.DataFrame(X_oversampled, columns=datosML.drop("Resultado", axis=1).columns)
datosoversampled["Resultado"] = y_oversampled


In [9]:
y_oversampled.value_counts()

0    1035
1    1035
Name: Resultado, dtype: int64

# Pre modelos machine learning

In [10]:
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [11]:
X= X_oversampled
y= y_oversampled

In [12]:

validation_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    test_size=validation_size,
                                                                    random_state=seed)


## MODELO 1: LOGISTIC REGRESION

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [16]:
prediction= model.predict(X_test)


In [18]:
accuracyregression=  model.score(X_test, y_test)
accuracyregression

0.5603864734299517

In [20]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)

[0.52409639 0.56024096 0.51204819 0.4939759  0.52409639 0.54216867
 0.61818182 0.52121212 0.58181818 0.49090909]


## MODELO 2: RANDOM TREE

In [21]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

DecisionTreeRegressor()

In [22]:
dtrprediction= dtr.predict(X_test)

In [24]:
dtraccuracy= dtr.score (X_test, y_test)
dtraccuracy

0.3576595432606253

In [25]:
dtraccscores = cross_val_score(dtr, X, y, cv=5)
dtraccscores

array([-1.30123438, -0.84111007, -0.82230483, -0.03289735,  0.        ])

## MODELO 3: BAGGING

In [26]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier()
modelbag = BaggingClassifier(base_estimator=dtc, n_estimators=100, random_state=seed)

In [27]:
modelbag.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=7)

In [28]:
bagpred = modelbag.predict(X_test)
bagcuracy=modelbag.score(X_test, y_test)
bagcuracy

0.782608695652174

In [29]:
bagscores = cross_val_score(modelbag, X, y, cv=5)
bagscores

array([0.71014493, 0.58937198, 0.77536232, 0.66666667, 0.72222222])

In [30]:
mean_score = bagscores.mean()
print("Mean Accuracy:", mean_score)

Mean Accuracy: 0.6927536231884057


In [31]:
# pruebas extra para validar la fiabilidad

from sklearn.metrics import precision_score, recall_score, f1_score

In [33]:
#precision 
precision = precision_score(y_test, bagpred)
precision

0.7327586206896551

In [35]:
#recall

recall = recall_score(y_test, bagpred)
recall

0.8585858585858586

In [37]:
#f1 score
f1 = f1_score(y_test, bagpred)
f1

0.7906976744186046

## MODELO 4: Ada Boost

In [38]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)


AdaBoostClassifier()

In [39]:
adapred=adaboost.predict(X_test)


In [41]:
adaaccuracy=adaboost.score(X_test, y_test)
adaaccuracy

0.5893719806763285

In [42]:
adaboostscores = cross_val_score(adaboost, X, y, cv=5)
adaboostscores

array([0.4178744 , 0.3236715 , 0.50241546, 0.43478261, 0.48550725])

## MEJOR MODELO: BAGGING 

In [44]:
import joblib

joblib.dump(modelbag, 'new_model.pkl')

['new_model.pkl']

# FEATURE IMPORTANCE

In [46]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

In [47]:
bagging_model = BaggingRegressor(base_estimator=DecisionTreeRegressor())

In [49]:
bagging_model.fit(X_train, y_train)

BaggingRegressor(base_estimator=DecisionTreeRegressor())

In [51]:
#feature importance


feature_importances = []
for base_model in bagging_model.estimators_:
    feature_importances.append(base_model.feature_importances_)

In [52]:
average_importance = np.mean(feature_importances, axis=0)

In [64]:
sorted_indices = np.argsort(average_importance)[::-1]


array([0, 2, 3, 1], dtype=int64)

In [69]:
X_train_df = pd.DataFrame(X_train, columns=['MES', 'COMERCIAL_NUM', 'TELEOP_NUM', 'CANAL_NUM', 'tipodeobra'])


In [70]:
X_train_df

,MES,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
1238,1,7.0,9,NaN,1
450,5,6.0,1,NaN,5
1114,9,2.0,1,NaN,5
1539,10,1.0,1,NaN,5
751,2,3.0,6,NaN,5
...,...,...,...,...,...
211,1,1.0,7,NaN,1
1603,2,1.0,6,NaN,1
537,7,3.0,7,NaN,1
1220,1,1.0,4,NaN,4


In [63]:
sorted_features = X_train_df.columns[sorted_indices]


Index(['MES', 'TELEOP_NUM', 'CANAL_NUM', 'COMERCIAL_NUM'], dtype='object')

In [59]:
for feature, importance in zip(sorted_features, average_importance):
    print(f"{feature}: {importance}")

MES: 0.31388199843851006
TELEOP_NUM: 0.18950415491467099
CANAL_NUM: 0.2774010386433752
COMERCIAL_NUM: 0.21921280800344367
